# Tasks for data preparation

- ✅ Aggregate movement data by hour
- ✅ Join together all datasets
- ❌ Aggregate data by day

# Table of contents
### 1. Load the data
### 2. Transform data
### 3. Join the data
### 4. Export the data to .csv

## 1. Load the data

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
import io
import requests

In [7]:
# Load movements
url = "https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo/download/2019_verkehrszaehlungen_werte_fussgaenger_velo.csv"
s = requests.get(url).content
movements = pd.read_csv(io.StringIO(s.decode('utf-8')))

# Rename columns
new_column_names = ['Standort', 'Datum', 'VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT', 'Ost', 'Nord']
movements.columns = new_column_names

# Load metadata
meta = pd.read_csv('../data/GeoData_StadtZurich/data/taz.view_eco_standorte.csv')

# Load weather data
url = "https://data.stadt-zuerich.ch/dataset/ugz_meteodaten_stundenmittelwerte/download/ugz_ogd_meteo_h1_2019.csv"
s = requests.get(url).content
weather2023 = pd.read_csv(io.StringIO(s.decode('utf-8')))

# Load population data
population = pd.read_csv('../data/bev324od3243.csv')

## 2. Overview of the data

### Display movements data

In [8]:
# Format Datum
movements['Datum'] = pd.to_datetime(movements['Datum'])
# Remove minutes information 
movements['Datum'] = movements['Datum'].dt.floor('1h')
# Format the timestamps and put time and date it into extra columns
movements['Date'] = movements['Datum'].dt.strftime('%Y-%m-%d')
movements['Time'] = movements['Datum'].dt.strftime('%H:%M')
movements['Datetime'] = movements['Datum'].dt.strftime('%Y-%m-%d %H:%M')

# Overview movements
movements.head()

,Standort,Datum,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,Ost,Nord,Date,Time,Datetime
0,5003,2023-01-01,NaN,NaN,1.0,0.0,2682978,1248744,2023-01-01,00:00,2023-01-01 00:00
1,4257,2023-01-01,0.0,0.0,NaN,NaN,2681857,1251991,2023-01-01,00:00,2023-01-01 00:00
2,394,2023-01-01,NaN,NaN,1.0,1.0,2683573,1251687,2023-01-01,00:00,2023-01-01 00:00
3,2986,2023-01-01,0.0,0.0,NaN,NaN,2684578,1251966,2023-01-01,00:00,2023-01-01 00:00
4,3598,2023-01-01,0.0,0.0,NaN,NaN,2684006,1246566,2023-01-01,00:00,2023-01-01 00:00


### Display meta data

In [9]:
# Remove columns that are not needed
meta.drop(['bis', 'fk_zaehler', 'von'], axis = 1)

# Select most recent records for each 'id1'
meta = meta[meta['bis'].isnull()]

# Show
meta.head()

,abkuerzung,bezeichnung,bis,fk_zaehler,id1,richtung_in,richtung_out,von,objectid,korrekturfaktor,geometry
31,VZS_BASL,Baslerstrasse,NaN,Y2H20063173,3003,Bhf. Altstetten,Letzipark,20200716000000,32,1.00,POINT (2679767.2 1248986.1)
43,VZS_SCHE,Scheuchzerstrasse,NaN,Y2H19111477,2993,Innenstadt,Irchel,20200110000000,44,1.05,POINT (2683573.2 1248544.9)
58,VZS_BINZ,Binzmühlestrasse,NaN,Y2H21015036,4257,Glaubtenstrasse,Oerlikon,20221208000000,59,1.22,POINT (2681857 1251990.9)
59,FZS_LANS,Langstrasse (Unterführung Süd),NaN,U15G3063867,4260,Gleisanlagen,Limmat,20230211000000,60,1.00,POINT (2682347.8 1248427.2)
66,FZS_MILI,Militärbrücke,NaN,U15G3063864,20,Löwenplatz,Langstrasse,20130711000000,67,0.58,POINT (2682689 1247734.9)


### Display weather data
The weather data is available for three different locations `['Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse', 'Zch_Rosengartenstrasse']`. We will need only one of these.

1. Check for missing data:

In [10]:
# Print the number of missing values in each column
print(weather2023.isnull().sum())

Datum           0
Standort        0
Parameter       0
Intervall       0
Einheit         0
Wert         2687
Status          0
dtype: int64


Next analyze what data is missing... are there missing values for every location?

In [11]:
# Count total number of row per Standort
print('Number of records per Standort: \n', '----------------------------------')
print(weather2023['Standort'].value_counts())
# Select only rows with missing values
missing_weather = weather2023[weather2023.isnull().any(axis=1)]
print('')
# Print the number of missing values in each column
print('Number of missing values per Standort: \n', '----------------------------------')
print(missing_weather[['Standort', 'Parameter']].groupby('Standort').count())


Number of records per Standort: 
 ----------------------------------
Standort
Zch_Stampfenbachstrasse    70072
Zch_Schimmelstrasse        61313
Zch_Rosengartenstrasse     61313
Name: count, dtype: int64

Number of missing values per Standort: 
 ----------------------------------
                         Parameter
Standort                          
Zch_Rosengartenstrasse        1073
Zch_Schimmelstrasse           1579
Zch_Stampfenbachstrasse         35


We can conclude that Standort Stampfenbachstrasse has the most recorded values and also the least missing values. So we select `Standort == 'Zch_Stampfenbachstrasse'`. Also we will use a **simple imputation method** for dealing with the remaining missing data by picking a random value from the same day of measurement.

In [12]:
# Filter weather data for rows where 'Standort' is 'Zch_Stampfenbachstrasse'
MissingWeatherStampfenbach = weather2023[weather2023['Standort'] == 'Zch_Stampfenbachstrasse']
# Identify dates with missing values
missing_dates = MissingWeatherStampfenbach[MissingWeatherStampfenbach.isnull().any(axis=1)]["Datum"].unique()

In [13]:
# Fit the encoder on the data
cols = ['Standort', 'Datum', 'Parameter', 'Wert']

In [14]:
# Write a function that pics a random value from the same day 
# and same parameter if there is a missing value
def impute_missing_values(df):
    for index, row in df.iterrows():
        if pd.isnull(row['Wert']):
            # Get all rows with the same moment (datetime) and parameter and that are not missing
            same_moment = df[(df['Datum'] == row['Datum']) & 
                          (df['Parameter'] == row['Parameter']) & 
                          (df['Wert'].notnull())]
            # If there are rows within the same moment and parameter
            if len(same_moment) > 0:
                random_value = same_moment.sample(1)
                df.at[index, 'Wert'] = random_value['Wert'].values[0]
            # If there are no rows with the same date and parameter
            else:
                # Get all rows within the same day and the same parameter
                # Convert the 'Datum' to date only format and discard the time
                same_day = df[(pd.to_datetime(df['Datum']).dt.date == pd.to_datetime(row['Datum']).date()) & 
                          (df['Parameter'] == row['Parameter']) & 
                          (df['Wert'].notnull())]
                # If there are rows with the same date and parameter
                if len(same_day) > 0:
                    random_value = same_day.sample(1)
                    df.at[index, 'Wert'] = random_value['Wert'].values[0]
                # If there are no rows with the same date and parameter
                else:
                    # Get all rows with the same parameter the day before
                    day_before = df[
                              (pd.to_datetime(df['Datum']).dt.date == (pd.to_datetime(row['Datum']).date() - pd.DateOffset(1)).date() ) & 
                              (df['Parameter'] == row['Parameter']) & 
                              (df['Wert'].notnull())]
                    # If there are rows within the same data (+- 1 days) and parameter
                    if len(day_before) > 0:
                        random_value = day_before.sample(1)
                        df.at[index, 'Wert'] = random_value['Wert'].values[0]
                    else:
                        # Raise an error if there are no rows with the same parameter in two days before and after
                        raise ValueError('There are no rows with the same parameter in two days before and after')  
    return df

# Relevant missing rows
missing_weather = weather2023[weather2023['Datum'].isin(missing_dates)][cols].drop('Standort', axis=1)

# Set a seed for reproducibility
np.random.seed(123)

# Impute missing values
weather2023_imputed = impute_missing_values(weather2023)

# Check if there are still missing values
print('Missing values after imputation: \n', '----------------------------------')
print(weather2023_imputed.isnull().sum(), '\n')

# Show the rows with missing values
print('Row with still missing values: \n', '----------------------------------')
print(weather2023_imputed[weather2023_imputed.isnull().any(axis=1)].drop_duplicates())

Missing values after imputation: 
 ----------------------------------
Datum        0
Standort     0
Parameter    0
Intervall    0
Einheit      0
Wert         0
Status       0
dtype: int64 

Row with still missing values: 
 ----------------------------------
Empty DataFrame
Columns: [Datum, Standort, Parameter, Intervall, Einheit, Wert, Status]
Index: []


### Display population data

In [15]:
population.tail()

,StichtagDatJahr,AnzBestWir
118,2019,434008
119,2020,434736
120,2021,436332
121,2022,443037
122,2023,447082


## 2. Transform data

### Long to wide transformation for weather data

In [16]:
# Transform weather data into wide format
weather2023['Param_Unit'] = weather2023['Parameter'] + ' [' + weather2023['Einheit'] + ']'
wide_weather2023 = weather2023.pivot_table(index=['Datum'], columns='Param_Unit', values='Wert').reset_index()

# Display the wide format DataFrame
wide_weather2023.head()

Param_Unit,Datum,Hr [%Hr],RainDur [min],StrGlo [W/m2],T [°C],WD [°],WVs [m/s],WVv [m/s],p [hPa]
0,2023-01-01T00:00+0100,72.303333,0.0,0.01,11.883333,133.663333,1.216667,0.873333,972.443333
1,2023-01-01T01:00+0100,63.653333,0.0,0.02,13.903333,187.633333,1.870000,1.490000,972.676667
2,2023-01-01T02:00+0100,69.730000,0.0,0.02,12.280000,209.916667,1.203333,0.593333,972.620000
3,2023-01-01T03:00+0100,74.836667,0.0,0.02,10.930000,204.703333,0.960000,0.670000,972.910000
4,2023-01-01T04:00+0100,69.076667,0.0,0.02,12.260000,186.096667,1.743333,1.420000,972.956667


In [17]:
# Format Datum
wide_weather2023['Datum'] = pd.to_datetime(wide_weather2023['Datum'])
# Remove minutes information 
wide_weather2023['Datum'] = wide_weather2023['Datum'].dt.floor('1h')
# Format the timestamps and put time and date it into extra columns
wide_weather2023['Year'] = wide_weather2023['Datum'].dt.year
wide_weather2023['Date'] = wide_weather2023['Datum'].dt.strftime('%Y-%m-%d')
wide_weather2023['Time'] = wide_weather2023['Datum'].dt.strftime('%H:%M')
wide_weather2023['Datetime'] = wide_weather2023['Datum'].dt.strftime('%Y-%m-%d %H:%M')

# Overview
wide_weather2023.head()

Param_Unit,Datum,Hr [%Hr],RainDur [min],StrGlo [W/m2],T [°C],WD [°],WVs [m/s],WVv [m/s],p [hPa],Year,Date,Time,Datetime
0,2023-01-01 00:00:00+01:00,72.303333,0.0,0.01,11.883333,133.663333,1.216667,0.873333,972.443333,2023,2023-01-01,00:00,2023-01-01 00:00
1,2023-01-01 01:00:00+01:00,63.653333,0.0,0.02,13.903333,187.633333,1.870000,1.490000,972.676667,2023,2023-01-01,01:00,2023-01-01 01:00
2,2023-01-01 02:00:00+01:00,69.730000,0.0,0.02,12.280000,209.916667,1.203333,0.593333,972.620000,2023,2023-01-01,02:00,2023-01-01 02:00
3,2023-01-01 03:00:00+01:00,74.836667,0.0,0.02,10.930000,204.703333,0.960000,0.670000,972.910000,2023,2023-01-01,03:00,2023-01-01 03:00
4,2023-01-01 04:00:00+01:00,69.076667,0.0,0.02,12.260000,186.096667,1.743333,1.420000,972.956667,2023,2023-01-01,04:00,2023-01-01 04:00


In [18]:
# Are there any missing values?
wide_weather2023.isnull().sum()

Param_Unit
Datum            0
Hr [%Hr]         0
RainDur [min]    0
StrGlo [W/m2]    0
T [°C]           0
WD [°]           0
WVs [m/s]        0
WVv [m/s]        0
p [hPa]          0
Year             0
Date             0
Time             0
Datetime         0
dtype: int64

### Aggregate movement data per hour

In [19]:
movements.head()
# Aggregate based on 'Standort' and 'Time', sum 'VELO_IN', and get max 'Ost'
movements_hourly = movements.groupby(['Standort', 'Date', 'Time', 'Datetime']).agg({'VELO_IN' : 'sum',
                                                                        'VELO_OUT' : 'sum',
                                                                        'FUSS_IN' : 'sum',
                                                                        'FUSS_OUT' : 'sum',
                                                                        'Ost' : 'max',
                                                                        'Nord' : 'max'})

In [20]:
movements_hourly.head(10)

VELO_IN  VELO_OUT  FUSS_IN  \
Standort Date       Time  Datetime                                       
20       2023-01-01 00:00 2023-01-01 00:00      0.0       0.0     46.0   
                    01:00 2023-01-01 01:00      0.0       0.0     43.0   
                    02:00 2023-01-01 02:00      0.0       0.0     36.0   
                    03:00 2023-01-01 03:00      0.0       0.0     22.0   
                    04:00 2023-01-01 04:00      0.0       0.0     11.0   
                    05:00 2023-01-01 05:00      0.0       0.0     18.0   
                    06:00 2023-01-01 06:00      0.0       0.0     19.0   
                    07:00 2023-01-01 07:00      0.0       0.0      7.0   
                    08:00 2023-01-01 08:00      0.0       0.0      4.0   
                    09:00 2023-01-01 09:00      0.0       0.0     14.0   

                                            FUSS_OUT      Ost     Nord  
Standort Date       Time  Datetime                                      
20       2023-01-01 00:00 2023-01-01 00:00      31.0  2682689  1247735  
                    01:00 2023-01-01 01:00      94.0  2682689  1247735  
                    02:00 2023-01-01 02:00      27.0  2682689  1247735  
                    03:00 2023-01-01 03:00      27.0  2682689  1247735  
                    04:00 2023-01-01 04:00      33.0  2682689  1247735  
                    05:00 2023-01-01 05:00      14.0  2682689  1247735  
                    06:00 2023-01-01 06:00       6.0  2682689  1247735  
                    07:00 2023-01-01 07:00       0.0  2682689  1247735  
                    08:00 2023-01-01 08:00       6.0  2682689  1247735  
                    09:00 2023-01-01 09:00       1.0  2682689  1247735

In [21]:
# Are there any missing data in the movements DataFrame?
print('Missing values in the movements DataFrame: \n', '----------------------------------')
movements_hourly.isnull().sum()

Missing values in the movements DataFrame: 
 ----------------------------------


VELO_IN     0
VELO_OUT    0
FUSS_IN     0
FUSS_OUT    0
Ost         0
Nord        0
dtype: int64

## 3. Join the data

In [22]:
# Join movements and weather
df_agg_hourly = pd.merge(movements_hourly.reset_index(), 
                         wide_weather2023.drop(['Datum', 'Date', 'Time'], axis = 1), 
                         left_on = 'Datetime', 
                         right_on = 'Datetime', 
                         how = 'left')

# Join population data
df_agg_hourly = pd.merge(df_agg_hourly, 
                         population,
                         left_on = 'Year', 
                         right_on = 'StichtagDatJahr', 
                         how = 'left')

# Remove 'StichtagDatJahr'
df_agg_hourly = df_agg_hourly.drop('StichtagDatJahr', axis = 1)

# Join meta data
df_agg_hourly = pd.merge(df_agg_hourly, 
                         meta.drop(['bis', 'von', 'abkuerzung', 'objectid'], axis = 1),
                         left_on = 'Standort',
                         right_on = 'id1',
                         how = 'inner' # remove obs without metadata
                         )

# Remove 'id1'
df_agg_hourly = df_agg_hourly.drop('id1', axis = 1)

# Print dimensions
print('Dataframe dimension: ', df_agg_hourly.shape)

# Display
df_agg_hourly.head()


Dataframe dimension:  (220935, 26)


,Standort,Date,Time,Datetime,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,Ost,Nord,...,WVv [m/s],p [hPa],Year,AnzBestWir,bezeichnung,fk_zaehler,richtung_in,richtung_out,korrekturfaktor,geometry
0,20,2023-01-01,00:00,2023-01-01 00:00,0.0,0.0,46.0,31.0,2682689,1247735,...,0.873333,972.443333,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
1,20,2023-01-01,01:00,2023-01-01 01:00,0.0,0.0,43.0,94.0,2682689,1247735,...,1.490000,972.676667,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
2,20,2023-01-01,02:00,2023-01-01 02:00,0.0,0.0,36.0,27.0,2682689,1247735,...,0.593333,972.620000,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
3,20,2023-01-01,03:00,2023-01-01 03:00,0.0,0.0,22.0,27.0,2682689,1247735,...,0.670000,972.910000,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
4,20,2023-01-01,04:00,2023-01-01 04:00,0.0,0.0,11.0,33.0,2682689,1247735,...,1.420000,972.956667,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)


4. ## Deal with missing data
Next analyze if there is any missing data

In [23]:
# Analyze if there is misisng data
# Only show columns with missing data
df_agg_hourly.isnull().sum()

Standort              0
Date                  0
Time                  0
Datetime              0
VELO_IN               0
VELO_OUT              0
FUSS_IN               0
FUSS_OUT              0
Ost                   0
Nord                  0
Hr [%Hr]             27
RainDur [min]        27
StrGlo [W/m2]        27
T [°C]               27
WD [°]               27
WVs [m/s]            27
WVv [m/s]            27
p [hPa]              27
Year                 27
AnzBestWir           27
bezeichnung           0
fk_zaehler            0
richtung_in           0
richtung_out       3792
korrekturfaktor       0
geometry              0
dtype: int64

### Missing values in [`richtung_out`]
We conclude that missing values in `richtunng_out` are missing at random (MAR) because for `Standort = 4267`(Central) data is only beeing recorded one way (Limmatquai-->Central).

In [24]:
# Show rows where richtung_out is missing
missing_richtung = df_agg_hourly[df_agg_hourly['richtung_out'].isnull()]
# Only select attributes that are relevant [Standort, bezeichnung, richtung_in, richtung_out]
# Select only distinct rows for the selected attributes
missing_richtung[['Standort', 'bezeichnung', 'richtung_in', 'richtung_out']].drop_duplicates()

,Standort,bezeichnung,richtung_in,richtung_out
204832,4267,Limmatquai --> Central,Central,NaN


### Missing values in Weather data


Analysis shows that all the 27 missing data belongs to Date = '2023-12-31' at Time = '23:00'. This is because the movement data has one more observation that the weather data.
For this reason we will simply drop these records.

In [25]:
# Show rows where Hr [%Hr] is missing
missing_hr = df_agg_hourly[df_agg_hourly['Hr [%Hr]'].isnull()]
# Only select attributes that are relevant [Standort, bezeichnung, 'Hr [%Hr]']
missing_hr[['Standort', 'bezeichnung', 'Datetime', 'Hr [%Hr]', 'RainDur [min]', 'StrGlo [W/m2]', 'T [°C]']]

,Standort,bezeichnung,Datetime,Hr [%Hr],RainDur [min],StrGlo [W/m2],T [°C]
12622,732,Hardbrücke Nord (Seite Altstetten),2023-12-31 23:00,NaN,NaN,NaN,NaN
21381,1037,Hardbrücke Süd (Seite HB),2023-12-31 23:00,NaN,NaN,NaN,NaN
35972,2982,Limmatquai --> Bellevue,2023-12-31 23:00,NaN,NaN,NaN,NaN
44731,2986,Andreasstrasse,2023-12-31 23:00,NaN,NaN,NaN,NaN
53490,2989,Langstrasse (Unterführung Nord),2023-12-31 23:00,NaN,NaN,NaN,NaN
62249,2991,Talstrasse,2023-12-31 23:00,NaN,NaN,NaN,NaN
70864,2993,Scheuchzerstrasse,2023-12-31 23:00,NaN,NaN,NaN,NaN
79623,2997,Lux-Guyer-Weg,2023-12-31 23:00,NaN,NaN,NaN,NaN
90182,3003,Baslerstrasse,2023-12-31 23:00,NaN,NaN,NaN,NaN
98940,3012,Bucheggplatz,2023-12-31 23:00,NaN,NaN,NaN,NaN


In [26]:
# Drop all rows with 'Datetime' == '2023-12-31 23:00' 
df_agg_hourly = df_agg_hourly[df_agg_hourly['Datetime'] != '2023-12-31 23:00']

In [27]:
# Analyze if there is misisng data
# Only show columns with missing data
df_agg_hourly.isnull().sum()

Standort              0
Date                  0
Time                  0
Datetime              0
VELO_IN               0
VELO_OUT              0
FUSS_IN               0
FUSS_OUT              0
Ost                   0
Nord                  0
Hr [%Hr]              0
RainDur [min]         0
StrGlo [W/m2]         0
T [°C]                0
WD [°]                0
WVs [m/s]             0
WVv [m/s]             0
p [hPa]               0
Year                  0
AnzBestWir            0
bezeichnung           0
fk_zaehler            0
richtung_in           0
richtung_out       3791
korrekturfaktor       0
geometry              0
dtype: int64

In [28]:
# Add up the total number of bikes and pedestrians
df_agg_hourly['bike_tot'] = df_agg_hourly['VELO_IN'] + df_agg_hourly['VELO_OUT']
df_agg_hourly['ped_tot'] = df_agg_hourly['FUSS_IN'] + df_agg_hourly['FUSS_OUT']

# Keep only relevant columns
df_agg_relevant = df_agg_hourly.drop(['VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT', 'Ost', 'Nord', 'fk_zaehler', 'richtung_in', 'richtung_out', 'korrekturfaktor', 'geometry'], axis = 1)

# 4. Export the data

In [30]:
# Export as csv
df_agg_relevant.to_csv('../results/df_agg_hourly_2019.csv', index=False)